# Exercise 2

##### In the second exercise we examined the privileged and unprivileged groups that were identified by MDSS. 
##### For each of groups we measured its bias and compared it to a group that has the opposite race or sex. 

#####     -privileged (Female, Caucasian)
#####     -unprivileged (Male, Non-Caucasian) 

##### In our case, we initially analyzed sex, focusing on "Female" and "Male". 
##### Subsequently, we explored racial categories, we took "Caucasian" and "Non-Caucasian", with an additional emphasis on age "Less than 25". 
##### Finally, we conducted a comparative evaluation.

In [2]:
import itertools

import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric, MDSSClassificationMetric
from aif360.detectors import bias_scan

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

In [3]:
dataset_orig = load_preproc_data_compas()

female_group = [{'sex': 1}]
male_group = [{'sex': 0}]

In [4]:
dataset_orig_df = pd.DataFrame(dataset_orig.features, columns=dataset_orig.feature_names)

age_cat = np.argmax(dataset_orig_df[['age_cat=Less than 25']].values, axis=1).reshape(-1, 1)
priors_count = np.argmax(dataset_orig_df[['priors_count=0', 'priors_count=1 to 3',
                                          'priors_count=More than 3']].values, axis=1).reshape(-1, 1)
c_charge_degree = np.argmax(dataset_orig_df[['c_charge_degree=M', 'c_charge_degree=F']].values, axis=1).reshape(-1, 1)

features = np.concatenate((dataset_orig_df[['sex', 'race']].values, age_cat, priors_count,
                           c_charge_degree, dataset_orig.labels), axis=1)
feature_names = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']

In [5]:
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,two_year_recid
0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,2.0,1.0,1.0
2,0.0,1.0,0.0,2.0,1.0,1.0
3,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0


### Training

In [6]:
from aif360.datasets import StandardDataset
dataset = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race'],
                 privileged_classes=[[1], [1]],
                 instance_weights_name=None)

In [7]:
dataset_orig_train, dataset_orig_test = dataset.split([0.7], shuffle=True, seed=0)

### Difference in mean outcomes between unprivileged and privileged Sex groups

In [8]:
metric_train = BinaryLabelDatasetMetric(dataset_orig_train,
                             unprivileged_groups=male_group,
                             privileged_groups=female_group)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train.mean_difference())
metric_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=male_group,
                             privileged_groups=female_group)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test.mean_difference())


Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.124496
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.159410


It shows that overall Females in the dataset have a lower observed recidivism them Males.

If we train a classifier, the model is likely to pick up this bias in the dataset

In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

LogisticRegression(random_state=0)

##### Note that the probability scores we use are the probabilities of the favorable label, which is 0 in this case.

In [10]:
clf.classes_

array([0., 1.])

##### Predictions reflect the probability of a favorable outcome (i.e. no recidivism).

In [11]:
dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

In [12]:
df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,observed,probabilities
0,1.0,1.0,0.0,2.0,1.0,1.0,0.442952
1,1.0,0.0,0.0,0.0,1.0,0.0,0.690520
2,0.0,1.0,0.0,1.0,1.0,0.0,0.538732
3,0.0,0.0,0.0,2.0,1.0,1.0,0.301264
4,0.0,1.0,0.0,1.0,0.0,1.0,0.618771


##### We created another structured dataset as the classified dataset by assigning the predicted probabilities to the scores attribute.

In [13]:
dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels

### Bias scoring

In [14]:
test_df = dataset_bias_test.convert_to_dataframe()[0]
test_df['model_not_recid'] = dataset_bias_test.scores.flatten()
test_df['observed_not_recid'] = 1 - test_df['two_year_recid']
test_df

,sex,race,age_cat,priors_count,c_charge_degree,two_year_recid,model_not_recid,observed_not_recid
2479,1.0,1.0,0.0,2.0,1.0,1.0,0.442952,0.0
3574,1.0,0.0,0.0,0.0,1.0,0.0,0.690520,1.0
513,0.0,1.0,0.0,1.0,1.0,0.0,0.538732,1.0
1725,0.0,0.0,0.0,2.0,1.0,1.0,0.301264,0.0
96,0.0,1.0,0.0,1.0,0.0,1.0,0.618771,0.0
...,...,...,...,...,...,...,...,...
4931,0.0,1.0,0.0,1.0,1.0,0.0,0.538732,1.0
3264,0.0,0.0,0.0,0.0,1.0,1.0,0.625015,0.0
1653,0.0,0.0,0.0,1.0,1.0,0.0,0.458795,1.0
2607,1.0,1.0,0.0,0.0,1.0,1.0,0.754542,0.0


In [15]:
# Females actual vs predicted rates of positive label
test_df[test_df.sex == 1][['model_not_recid','observed_not_recid']].mean()

model_not_recid       0.619455
observed_not_recid    0.657051
dtype: float64

#### Since model average predictions for the positive label is lower than the observed average by a substantial amount about 4%, 
#### ((0.619455-0.657051)*100) - the female group is most likely unprivileged.

In [16]:
# Males actual vs predicted rates of positive label
test_df[test_df.sex == 0][['model_not_recid','observed_not_recid']].mean()

model_not_recid       0.509450
observed_not_recid    0.497642
dtype: float64

#### Since model average predictions for the positive label is greater than the observed average by a small amount about 1.18% 
#### ((0.509450-0.497642)*100) - the male group could be privileged.

## Further, we create an instance of the MDSS Classification Metric and assess the apriori defined privileged and unprivileged groups; females and males respectively. 

By apriori defining the male group as unprivileged, we are saying we expect that the model's predictions is systematically lower than the actual observation.

By apriori defining the female group as privileged, we are saying we expect that the model's predictions is systematically higher than the actual observation.

From our mini-analysis above, we know that these hypothesis are unlikely to be true 

In [17]:
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=male_group,
                                           privileged_groups=female_group)

In [18]:
# We are asking the question:
# Is there evidence that the hypothesized privileged group is actually privileged?

female_privileged_score = mdss_classified.score_groups(privileged=True)
female_privileged_score

-0.0

By having a score very close to zero, mdss bias score is informing us that there is no evidence from the data that our hypothesis of the female group being privileged is true.

In [19]:
# We are asking the question:
# Is there evidence that the hypothesized unprivileged group is actually unprivileged?

male_unprivileged_score = mdss_classified.score_groups(privileged=False)
male_unprivileged_score

-0.0

By having a score very close zero, mdss bias score is informing us that there is no evidence from the data to support our hypothesis of the male group being unprivileged is true.

### We can flip our initial hypothesis and check if the male group is privileged or the female group is unprivileged.

In [20]:
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=female_group,
                                           privileged_groups=male_group)

In [21]:
male_privileged_score = mdss_classified.score_groups(privileged=True)
male_privileged_score

0.389

By having a positive score, mdss bias score is informing us that there is evidence from the data that our hypothesis of the male group being privileged is true.

In [22]:
female_unprivileged_score = mdss_classified.score_groups(privileged=False)
female_unprivileged_score

1.0382

By having a positive score, mdss bias score is informing us that there is evidence from the data to support our hypothesis of the female group being unprivileged is true.

By taking into account the size of the group and the magnitude of the deviation, mdss bias core has been able to tell us the following about the male and female groups:
- There is no evidence that the female group is privileged.
- There is no evidence that the male group is unprivileged.
- There is evidence that the male group is privileged.
- There is evidence that the female is unprivileged.

Keep in mind that these results for sex are based on the age category beingless than 25. As such, all results are based on people with that age grouping in mind.

### Repeat all the steps for Race groups

In [23]:
dataset_orig = load_preproc_data_compas()

caucasian = [{'race': 1}]
non_caucasian = [{'race': 0}]

In [24]:
dataset_orig_df = pd.DataFrame(dataset_orig.features, columns=dataset_orig.feature_names)

age_cat = np.argmax(dataset_orig_df[['age_cat=Less than 25']].values, axis=1).reshape(-1, 1)
priors_count = np.argmax(dataset_orig_df[['priors_count=0', 'priors_count=1 to 3',
                                          'priors_count=More than 3']].values, axis=1).reshape(-1, 1)
c_charge_degree = np.argmax(dataset_orig_df[['c_charge_degree=M', 'c_charge_degree=F']].values, axis=1).reshape(-1, 1)

features = np.concatenate((dataset_orig_df[['sex', 'race']].values, age_cat, priors_count,
                           c_charge_degree, dataset_orig.labels), axis=1)
feature_names = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']

In [25]:
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,two_year_recid
0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,2.0,1.0,1.0
2,0.0,1.0,0.0,2.0,1.0,1.0
3,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0


In [26]:
from aif360.datasets import StandardDataset
dataset = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race'],
                 privileged_classes=[[1], [1]],
                 instance_weights_name=None)

In [27]:
dataset_orig_train, dataset_orig_test = dataset.split([0.7], shuffle=True, seed=0)

In [32]:
metric_train = BinaryLabelDatasetMetric(dataset_orig_train,
                             unprivileged_groups=non_caucasian,
                             privileged_groups=caucasian)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train.mean_difference())
metric_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=non_caucasian,
                             privileged_groups=caucasian)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test.mean_difference())


Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.132446
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.131891


In [33]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

LogisticRegression(random_state=0)

In [34]:
clf.classes_

array([0., 1.])

In [35]:
dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

In [36]:
df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,observed,probabilities
0,1.0,1.0,0.0,2.0,1.0,1.0,0.442952
1,1.0,0.0,0.0,0.0,1.0,0.0,0.690520
2,0.0,1.0,0.0,1.0,1.0,0.0,0.538732
3,0.0,0.0,0.0,2.0,1.0,1.0,0.301264
4,0.0,1.0,0.0,1.0,0.0,1.0,0.618771


In [37]:
dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels

In [38]:
test_df = dataset_bias_test.convert_to_dataframe()[0]
test_df['model_not_recid'] = dataset_bias_test.scores.flatten()
test_df['observed_not_recid'] = 1 - test_df['two_year_recid']
test_df

,sex,race,age_cat,priors_count,c_charge_degree,two_year_recid,model_not_recid,observed_not_recid
2479,1.0,1.0,0.0,2.0,1.0,1.0,0.442952,0.0
3574,1.0,0.0,0.0,0.0,1.0,0.0,0.690520,1.0
513,0.0,1.0,0.0,1.0,1.0,0.0,0.538732,1.0
1725,0.0,0.0,0.0,2.0,1.0,1.0,0.301264,0.0
96,0.0,1.0,0.0,1.0,0.0,1.0,0.618771,0.0
...,...,...,...,...,...,...,...,...
4931,0.0,1.0,0.0,1.0,1.0,0.0,0.538732,1.0
3264,0.0,0.0,0.0,0.0,1.0,1.0,0.625015,0.0
1653,0.0,0.0,0.0,1.0,1.0,0.0,0.458795,1.0
2607,1.0,1.0,0.0,0.0,1.0,1.0,0.754542,0.0


In [39]:
# Caucasian actual vs predicted rates of positive label
test_df[test_df.race == 1][['model_not_recid','observed_not_recid']].mean()

model_not_recid       0.609058
observed_not_recid    0.608974
dtype: float64

#### Since model average predictions for the positive label is slightly higher than the observed average by very small amount which is about 0.84%, the Caucasian group is a bit likely privileged.

In [47]:
# Non-caucasian actual vs predicted rates of positive label
test_df[test_df.race == 0][['model_not_recid','observed_not_recid']].mean()

model_not_recid       0.480456
observed_not_recid    0.477083
dtype: float64

#### Since model average predictions for the positive label is slightly greater than the observed average by a super small amount about 0.38%, the Non-Caucasian group might be privileged.

In [41]:
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=non_caucasian,
                                           privileged_groups=caucasian)

In [42]:
# We are asking the question:
# Is there evidence that the hypothesized privileged group is actually privileged?

caucasian_privileged_score = mdss_classified.score_groups(privileged=True)
caucasian_privileged_score

0.0

#### By having a score very close to zero (in our case actually is zero), the mdss bias score is informing us that there is no evidence from the data that our hypothesis of the Caucasian group being privileged is true.

In [43]:
# We are asking the question:
# Is there evidence that the hypothesized unprivileged group is actually unprivileged?

non_caucasian_unprivileged_score = mdss_classified.score_groups(privileged=False)
non_caucasian_unprivileged_score

-0.0

#### Hese is also a score is zero, which means that there is no evidence from the data to support the hypothesis that non-Caucasian group being unprivileged is true.

In [44]:
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=caucasian,
                                           privileged_groups=non_caucasian)

In [45]:
non_caucasian_privileged_score = mdss_classified.score_groups(privileged=True)
non_caucasian_privileged_score

0.0238

#### Here, even if we have a positive score, it's marginally above zero, indicating that the data does not substantially support our hypothesis about the Caucasian group being privileged.

In [46]:
caucasian_unprivileged_score = mdss_classified.score_groups(privileged=False)
caucasian_unprivileged_score

-0.0

#### Here we can observe that score is zero that means there is No evidence from the data to support our hypothesis of the Caucasian group being unprivileged is true.

Keep in mind that these results for race are based on the age category beingless than 25. As such, all results are based on people with that age grouping in mind.